# Calculate stats for each numerical experiment for each ensembles

- First, calculate stat for each ensemble
- Second, calculate mean stat over all ensembles

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import geopandas as gpd
from p_tqdm import p_map
from functools import partial
from datetime import date
from scipy import stats
os.chdir("..")
import tools.marineHeatWaves as mhw
import warnings
warnings.filterwarnings('ignore')

/work/xinchenhe_umass_edu/.conda/envs/laketemp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# data paths to simulated water temperature
a2w_full_sim_dir = "/nas/cee-hydro/laketemp_bias/simulations/a2w_full_sim"
a2w_cloud_sim_dir = "/nas/cee-hydro/laketemp_bias/simulations/a2w_cloud_sim"
lstm_full_sim_dir = "/nas/cee-hydro/laketemp_bias/simulations/lstm_full_sim"
lstm_cloud_sim_dir = "/nas/cee-hydro/laketemp_bias/simulations/lstm_cloud_sim"

In [3]:
train_period = pd.date_range("2003-01-01", "2017-12-31")
val_period = pd.date_range("2018-01-01", "2023-12-31")
total_period = pd.date_range("2003-01-01", "2023-12-31")

In [4]:
# load cci lakes
cci_lake_list = pd.read_csv("data/cci_lakes_hydrolake_depth.csv")["CCI ID"].to_numpy()
cci_lakes = pd.read_csv("data/ESA_CCI_static_lake_mask_v2_1km_UoR_metadata_fv2.1_06Oct2021_4laketemp.csv", index_col=0).loc[cci_lake_list]
cci_lakes_gdf = gpd.GeoDataFrame(cci_lakes, geometry=gpd.points_from_xy(cci_lakes['LON CENTRE'], cci_lakes['LAT CENTRE']),
                                crs="epsg:4326")
cci_lakes_gdf.head()

,NAME,COUNTRY,LAT CENTRE,LON CENTRE,MAX DISTANCE TO LAND (KM),LAT MIN BOX,LAT MAX BOX,LON MIN BOX,LON MAX BOX,ID in GLOBOLAKES 1000 MASK,ID in CGLOPS MASK,geometry
CCI ID,,,,,,,,,,,,
799,Hawizeh marshes,Iraq;Iran Islamic Republic of,31.3792,47.7236,1.0,31.2042,31.4292,47.6042,47.7792,799;,799;,POINT (47.7236 31.3792)
3114,loch Ness,United Kingdom,57.3792,-4.3597,1.2,57.1042,57.4542,-4.7125,-4.2958,3114;,NaN,POINT (-4.3597 57.3792)
7889,lough Melvin,Ireland;United Kingdom,54.4208,-8.1264,1.3,54.3625,54.4958,-8.2875,-8.0542,7889;,NaN,POINT (-8.1264 54.4208)
2516,loch Lomond,United Kingdom,56.0708,-4.5792,1.5,55.9708,56.3458,-4.7542,-4.4792,2516;,NaN,POINT (-4.5792 56.0708)
12262,loch Leven,United Kingdom,56.2042,-3.3792,1.5,56.1375,56.2625,-3.4542,-3.2958,12262;,NaN,POINT (-3.3792 56.2042)


In [5]:
def cal_warmest_month_mean(df):
    """
    1. Find the warmest month for each year.
    2. Calculate the average temperature of the warmest months.
    """
    # monthly mean across df period
    monthly_temperature = df.resample("ME").mean()
    # for each year, find the warmest month mean temperature
    # then calculate mean across the df period
    warmest_month_temperature = monthly_temperature.groupby(monthly_temperature.index.year).max().mean()
    return warmest_month_temperature

def cal_coldest_month_mean(df):
    """
    1. Find the coldest month for each year.
    2. Calculate the average temperature of the coldest months.
    """
    # monthly mean across df period
    monthly_temperature = df.resample("ME").mean()
    # for each year, find the warmest month mean temperature
    # then calculate mean across the df period
    coldest_month_temperature = monthly_temperature.groupby(monthly_temperature.index.year).min().mean()
    return coldest_month_temperature

def cal_mhw_stats(df):
    '''
    A function to calculate the marine heat wave stats
    
    return number of heat wave events, averaged intensity, averaged duration. They are all averaged across ensembles.
    '''
    
    # from the df index and turn them into coordinal and save as variable t
    t = df.index.map(lambda x: x.toordinal()).to_numpy()
    
    # get the ensemble number
    # in our study, it's 10
    ensemble_num = len(df.columns)
    
    # we look at three major metrics
    n_events = [] # number of heat wave events
    avg_intensity = [] # averaged maximum intensity across all heat wave events
    avg_duration = [] # duration
    
    for i in range(ensemble_num):
        # calculate the heat wave stats for each ensemble
        mhws, clim = mhw.detect(t, df.iloc[:, i].to_numpy().ravel())
        n_events.append(mhws["n_events"])
        avg_intensity.append(np.mean(mhws["intensity_mean"]))
        avg_duration.append(np.mean(mhws["duration"]))
    
    # average across ensembles
    avg_n_events = np.mean(n_events)
    avg_avg_intensity = np.mean(avg_intensity)
    avg_avg_duration = np.mean(avg_duration)
    
    return avg_n_events, avg_avg_intensity, avg_avg_duration

In [6]:
def cal_ice_days(df, 
                 threshold = 0.76):
    '''
    A function to calculate number of days covered by ice
    
    Ice-cover: temperature < threshold
    
    Threshold is determined by the maximum RMSE during ice-covered period across study lakes
    '''
    # create a true/false dataframe
    df_ice = df <= threshold
    
    # sum on each year and calculate the mean across different ensembles
    ice_days = df_ice.groupby(df_ice.index.year).sum().mean()
    
    return ice_days

In [7]:
def cal_stat(lake_id, period, return_wl_df = False):
    # air2water, full
    a2w_full_sim = pd.read_csv(f"{a2w_full_sim_dir}/{lake_id}.csv", index_col=0, parse_dates=True).loc[period]
    a2w_full_sim_mean = a2w_full_sim.mean().mean()
    a2w_full_sim_warmest_month_mean = cal_warmest_month_mean(a2w_full_sim).mean()
    a2w_full_sim_coldest_month_mean = cal_coldest_month_mean(a2w_full_sim).mean()
    a2w_full_sim_ice_days_0 = cal_ice_days(a2w_full_sim, threshold=0).mean()
    a2w_full_sim_ice_days_076 = cal_ice_days(a2w_full_sim, threshold=0.76).mean()

    # air2water, cloud
    a2w_cloud_sim = pd.read_csv(f"{a2w_cloud_sim_dir}/{lake_id}.csv", index_col=0, parse_dates=True).loc[period]
    a2w_cloud_sim_mean = a2w_cloud_sim.mean().mean()
    a2w_cloud_sim_warmest_month_mean = cal_warmest_month_mean(a2w_cloud_sim).mean()
    a2w_cloud_sim_coldest_month_mean = cal_coldest_month_mean(a2w_cloud_sim).mean()
    a2w_cloud_sim_ice_days_0 = cal_ice_days(a2w_cloud_sim, threshold=0).mean()
    a2w_cloud_sim_ice_days_076 = cal_ice_days(a2w_cloud_sim, threshold=0.76).mean()

    # LSTM, full
    lstm_full_sim = pd.read_csv(f"{lstm_full_sim_dir}/{lake_id}.csv", index_col=0, parse_dates=True).loc[period]
    lstm_full_sim_mean = lstm_full_sim.mean().mean()
    lstm_full_sim_warmest_month_mean = cal_warmest_month_mean(lstm_full_sim).mean()
    lstm_full_sim_coldest_month_mean = cal_coldest_month_mean(lstm_full_sim).mean()
    lstm_full_sim_ice_days_0 = cal_ice_days(lstm_full_sim, threshold=0).mean()
    lstm_full_sim_ice_days_076 = cal_ice_days(lstm_full_sim, threshold=0.76).mean()

    # LSTM, cloud
    lstm_cloud_sim = pd.read_csv(f"{lstm_cloud_sim_dir}/{lake_id}.csv", index_col=0, parse_dates=True).loc[period]
    lstm_cloud_sim_mean = lstm_cloud_sim.mean().mean()
    lstm_cloud_sim_warmest_month_mean = cal_warmest_month_mean(lstm_cloud_sim).mean()
    lstm_cloud_sim_coldest_month_mean = cal_coldest_month_mean(lstm_cloud_sim).mean()
    lstm_cloud_sim_ice_days_0 = cal_ice_days(lstm_cloud_sim, threshold=0).mean()
    lstm_cloud_sim_ice_days_076 = cal_ice_days(lstm_cloud_sim, threshold=0.76).mean()
    
    
    # t-test between full and cloud for Air2Water
    a2w_pvalue_mean = stats.ttest_ind(a2w_full_sim.mean(), a2w_cloud_sim.mean(), nan_policy='omit').pvalue
    a2w_pvalue_warmest_month_mean = stats.ttest_ind(cal_warmest_month_mean(a2w_full_sim), cal_warmest_month_mean(a2w_cloud_sim), nan_policy='omit').pvalue
    a2w_pvalue_coldest_month_mean = stats.ttest_ind(cal_coldest_month_mean(a2w_full_sim), cal_coldest_month_mean(a2w_cloud_sim), nan_policy='omit').pvalue
    a2w_pvalue_ice_days_0 = stats.ttest_ind(cal_ice_days(a2w_full_sim, threshold=0), cal_ice_days(a2w_cloud_sim, threshold=0), nan_policy='omit').pvalue
    a2w_pvalue_ice_days_076 = stats.ttest_ind(cal_ice_days(a2w_full_sim, threshold=0.76), cal_ice_days(a2w_cloud_sim, threshold=0.76), nan_policy='omit').pvalue

    # t-test between full and cloud for LSTM
    lstm_pvalue_mean = stats.ttest_ind(lstm_full_sim.mean(), lstm_cloud_sim.mean(), nan_policy='omit').pvalue
    lstm_pvalue_warmest_month_mean = stats.ttest_ind(cal_warmest_month_mean(lstm_full_sim), cal_warmest_month_mean(lstm_cloud_sim), nan_policy='omit').pvalue
    lstm_pvalue_coldest_month_mean = stats.ttest_ind(cal_coldest_month_mean(lstm_full_sim), cal_coldest_month_mean(lstm_cloud_sim), nan_policy='omit').pvalue
    lstm_pvalue_ice_days_0 = stats.ttest_ind(cal_ice_days(lstm_full_sim, threshold=0), cal_ice_days(lstm_cloud_sim, threshold=0), nan_policy='omit').pvalue
    lstm_pvalue_ice_days_076 = stats.ttest_ind(cal_ice_days(lstm_full_sim, threshold=0.76), cal_ice_days(lstm_cloud_sim, threshold=0.76), nan_policy='omit').pvalue
    
    
    
    # Compute differences between cloud and full
    a2w_diff_mean = a2w_cloud_sim_mean - a2w_full_sim_mean
    a2w_diff_warmest_mean = a2w_cloud_sim_warmest_month_mean - a2w_full_sim_warmest_month_mean
    a2w_diff_ice_days_0 = a2w_cloud_sim_ice_days_0 - a2w_full_sim_ice_days_0
    a2w_diff_ice_days_076 = a2w_cloud_sim_ice_days_076 - a2w_full_sim_ice_days_076

    lstm_diff_mean = lstm_cloud_sim_mean - lstm_full_sim_mean
    lstm_diff_warmest_mean = lstm_cloud_sim_warmest_month_mean - lstm_full_sim_warmest_month_mean
    lstm_diff_ice_days_0 = lstm_cloud_sim_ice_days_0 - lstm_full_sim_ice_days_0
    lstm_diff_ice_days_076 = lstm_cloud_sim_ice_days_076 - lstm_full_sim_ice_days_076
    
    # Build stat dataframe
    stat_df = pd.DataFrame([[
        # Air2Water full
        a2w_full_sim_mean, a2w_full_sim_warmest_month_mean, a2w_full_sim_coldest_month_mean,
        a2w_full_sim_ice_days_0, a2w_full_sim_ice_days_076,

        # Air2Water cloud
        a2w_cloud_sim_mean, a2w_cloud_sim_warmest_month_mean, a2w_cloud_sim_coldest_month_mean,
        a2w_cloud_sim_ice_days_0, a2w_cloud_sim_ice_days_076,

        # Air2Water difference
        a2w_diff_mean, a2w_diff_warmest_mean, a2w_diff_ice_days_0, a2w_diff_ice_days_076,

        # LSTM full
        lstm_full_sim_mean, lstm_full_sim_warmest_month_mean, lstm_full_sim_coldest_month_mean,
        lstm_full_sim_ice_days_0, lstm_full_sim_ice_days_076,

        # LSTM cloud
        lstm_cloud_sim_mean, lstm_cloud_sim_warmest_month_mean, lstm_cloud_sim_coldest_month_mean,
        lstm_cloud_sim_ice_days_0, lstm_cloud_sim_ice_days_076,

        # LSTM difference
        lstm_diff_mean, lstm_diff_warmest_mean, lstm_diff_ice_days_0, lstm_diff_ice_days_076,

        # Air2Water p-values
        a2w_pvalue_mean, a2w_pvalue_warmest_month_mean, a2w_pvalue_coldest_month_mean,
        a2w_pvalue_ice_days_0, a2w_pvalue_ice_days_076,

        # LSTM p-values
        lstm_pvalue_mean, lstm_pvalue_warmest_month_mean, lstm_pvalue_coldest_month_mean,
        lstm_pvalue_ice_days_0, lstm_pvalue_ice_days_076,
    ]], index=[lake_id], columns=[
        # Air2Water full
        "a2w_mean", "a2w_warmest_mean", "a2w_coldest_mean", 
        "a2w_ice_days_0", "a2w_ice_days_076",

        # Air2Water cloud
        "a2w_cloud_mean", "a2w_cloud_warmest_mean", "a2w_cloud_coldest_mean",
        "a2w_cloud_ice_days_0", "a2w_cloud_ice_days_076",

        # Air2Water diff
        "a2w_diff_mean", "a2w_diff_warmest_mean", "a2w_diff_ice_days_0", "a2w_diff_ice_days_076",

        # LSTM full
        "lstm_mean", "lstm_warmest_mean", "lstm_coldest_mean",
        "lstm_ice_days_0", "lstm_ice_days_076",

        # LSTM cloud
        "lstm_cloud_mean", "lstm_cloud_warmest_mean", "lstm_cloud_coldest_mean",
        "lstm_cloud_ice_days_0", "lstm_cloud_ice_days_076",

        # LSTM diff
        "lstm_diff_mean", "lstm_diff_warmest_mean", "lstm_diff_ice_days_0", "lstm_diff_ice_days_076",

        # Air2Water p-values
        "a2w_pval_mean", "a2w_pval_warmest_mean", "a2w_pval_coldest_mean",
        "a2w_pval_ice_days_0", "a2w_pval_ice_days_076",

        # LSTM p-values
        "lstm_pval_mean", "lstm_pval_warmest_mean", "lstm_pval_coldest_mean",
        "lstm_pval_ice_days_0", "lstm_pval_ice_days_076"
    ])

    stat_df.index.name = "cci_lake_id"
    
    if return_wl_df == False:
        return stat_df
    else:
        return stat_df, a2w_full_sim, a2w_cloud_sim, lstm_full_sim, lstm_cloud_sim

In [8]:
def cal_stat_with_mhw(lake_id, period):
    stat_df, a2w_full_sim, a2w_cloud_sim, lstm_full_sim, lstm_cloud_sim = cal_stat(lake_id, period, return_wl_df=True)
    
    # some lakes cannot be derived with lake heat wave events
    a2w_full_sim = pd.read_csv(f"{a2w_full_sim_dir}/{lake_id}.csv", index_col=0, parse_dates=True).loc[period]
    try:
        a2w_full_sim_lhw_n_events, a2w_full_sim_lhw_intensity, a2w_full_sim_lhw_duration = cal_mhw_stats(a2w_full_sim)
        a2w_cloud_sim_lhw_n_events, a2w_cloud_sim_lhw_intensity, a2w_cloud_sim_lhw_duration = cal_mhw_stats(a2w_cloud_sim)
        lstm_full_sim_lhw_n_events, lstm_full_sim_lhw_intensity, lstm_full_sim_lhw_duration = cal_mhw_stats(lstm_full_sim)
        lstm_cloud_sim_lhw_n_events, lstm_cloud_sim_lhw_intensity, lstm_cloud_sim_lhw_duration = cal_mhw_stats(lstm_cloud_sim)
    except:
        a2w_full_sim_lhw_n_events, a2w_full_sim_lhw_intensity, a2w_full_sim_lhw_duration = np.nan, np.nan, np.nan
        a2w_cloud_sim_lhw_n_events, a2w_cloud_sim_lhw_intensity, a2w_cloud_sim_lhw_duration = np.nan, np.nan, np.nan
        lstm_full_sim_lhw_n_events, lstm_full_sim_lhw_intensity, lstm_full_sim_lhw_duration = np.nan, np.nan, np.nan
        lstm_cloud_sim_lhw_n_events, lstm_cloud_sim_lhw_intensity, lstm_cloud_sim_lhw_duration = np.nan, np.nan, np.nan
        
    # load to stat_df
    stat_df.loc[lake_id, 
                ["a2w_lhw_n_events", "a2w_lhw_intensity", "a2w_lhw_duration",
                 "a2w_cloud_lhw_n_events", "a2w_cloud_lhw_intensity", "a2w_cloud_lhw_duration",
                 "lstm_lhw_n_events", "lstm_lhw_intensity", "lstm_lhw_duration",
                 "lstm_cloud_lhw_n_events", "lstm_cloud_lhw_intensity", "lstm_cloud_lhw_duration"]] = [a2w_full_sim_lhw_n_events, a2w_full_sim_lhw_intensity, a2w_full_sim_lhw_duration,
                                                                                                       a2w_cloud_sim_lhw_n_events, a2w_cloud_sim_lhw_intensity, a2w_cloud_sim_lhw_duration,
                                                                                                       lstm_full_sim_lhw_n_events, lstm_full_sim_lhw_intensity, lstm_full_sim_lhw_duration,
                                                                                                       lstm_cloud_sim_lhw_n_events, lstm_cloud_sim_lhw_intensity, lstm_cloud_sim_lhw_duration,
                                                                                                      ]
    return stat_df

It looks like the heat wave metric does not apply to arctic lakes

In [9]:
train_stats = pd.concat(p_map(partial(cal_stat, period = train_period), cci_lake_list), axis = 0)
val_stats = pd.concat(p_map(partial(cal_stat, period = val_period), cci_lake_list), axis = 0)
train_stats.to_csv("data/sim_stats_trainperiod_ensemble_mean.csv")
val_stats.to_csv("data/sim_stats_valperiod_ensemble_mean.csv")
# total_stats = pd.concat(p_map(partial(cal_stat, period = total_period), cci_lake_list), axis = 0)

100%|██████████| 2016/2016 [00:12<00:00, 162.70it/s]


In [10]:
# train_stats = pd.concat(p_map(partial(cal_stat_with_mhw, period = train_period), cci_lake_list), axis = 0)
# val_stats = pd.concat(p_map(partial(cal_stat_with_mhw, period = val_period), cci_lake_list), axis = 0)
total_stats = pd.concat(p_map(partial(cal_stat_with_mhw, period = total_period), cci_lake_list), axis = 0)
total_stats.to_csv("data/sim_stats_totalperiod_ensemble_mean.csv")

100%|██████████| 2016/2016 [06:31<00:00,  5.15it/s]
